In [ ]:
!pip install -q python-docx transformers sqlite-utils

In [ ]:
import sqlite3
import pandas as pd
from docx import Document
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
def extract_tables_from_docx(docx_path):
    doc = Document(docx_path)
    tables = []
    for table in doc.tables:
        rows = []
        for row in table.rows:
            rows.append([cell.text.strip() for cell in row.cells])
        tables.append(pd.DataFrame(rows[1:], columns=rows[0]))
    return tables

In [ ]:
def populate_sqlite(tables, db_path="dicionario.db"):
    conn = sqlite3.connect(db_path)
    for i, df in enumerate(tables):
        table_name = f"tabela_{i}"
        df.to_sql(table_name, conn, if_exists="replace", index=False)
    return conn

In [ ]:
from google.colab import files
uploaded = files.upload()
docx_file = list(uploaded.keys())[0]

In [ ]:
tables = extract_tables_from_docx(docx_file)
conn = populate_sqlite(tables)
print(f"{len(tables)} tabelas inseridas no banco.")

In [ ]:
def get_schema(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql(query, conn)
    for t in tables['name']:
        df = pd.read_sql(f"SELECT * FROM {t} LIMIT 3", conn)
        print(f"📌 Tabela: {t}")
        print(df.head(), "\n")

In [ ]:
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
llm = pipeline("text2text-generation", model=model, tokenizer=tokenizer)



In [ ]:
def gerar_sql(pergunta, schema_hint=""):
    prompt = f"""
Você é um assistente que transforma perguntas em linguagem natural em comandos SQL para SQLite.
Contexto do banco de dados:
{schema_hint}

Pergunta: {pergunta}
Resposta:"""
    resposta = llm(prompt, max_new_tokens=128)[0]['generated_text']
    return resposta.strip()


In [ ]:
pergunta = "Qual a descrição do campo CNS_PROF da tabela NFCES110?"
schema_text = "\n".join(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)['name'])
sql = gerar_sql(pergunta, schema_hint=schema_text)
print("🔎 SQL gerado:", sql)

try:
    resultado = pd.read_sql(sql, conn)
    display(resultado)
except Exception as e:
    print("Erro ao executar SQL:", e)

In [ ]:
get_schema(conn)